In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torchdiffeq import odeint
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler

In [10]:
# Set device
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda:1


In [11]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [12]:
# Load the data
complete_data = pd.read_csv('../../data/raw/uav_hugging_face.csv')
incomplete_data = pd.read_csv('../../data/processed/uav_hugging_face_dropped_20.csv')

In [13]:
# Extract time and features
complete_time = complete_data['timestamp'].values
complete_features = complete_data[['tx', 'ty', 'tz']].values


In [14]:
incomplete_time = incomplete_data['timestamp'].values
incomplete_features = incomplete_data[['tx', 'ty', 'tz']].values


In [15]:
# Normalize the time to [0, 1]
time_scaler = MinMaxScaler()
complete_time_norm = time_scaler.fit_transform(complete_time.reshape(-1, 1)).flatten()


In [16]:
# Normalize features
feature_scaler = MinMaxScaler()
complete_features_norm = feature_scaler.fit_transform(complete_features)
incomplete_features_norm = feature_scaler.transform(incomplete_features)


In [17]:
# Create a mask for observed time points in the incomplete dataset
observed_mask = np.zeros(len(complete_time))
for t in incomplete_time:
    idx = np.where(complete_time == t)[0]
    if len(idx) > 0:
        observed_mask[idx[0]] = 1

In [18]:
# Convert to PyTorch tensors
complete_time_tensor = torch.tensor(complete_time_norm, dtype=torch.float32)
complete_features_tensor = torch.tensor(complete_features_norm, dtype=torch.float32)
incomplete_time_tensor = torch.tensor(time_scaler.transform(incomplete_time.reshape(-1, 1)).flatten(), dtype=torch.float32)
incomplete_features_tensor = torch.tensor(incomplete_features_norm, dtype=torch.float32)
observed_mask_tensor = torch.tensor(observed_mask, dtype=torch.float32)


In [19]:
# Define the Latent ODE model components
class LatentODEFunc(nn.Module):
    """ODE function for the latent dynamics"""
    def __init__(self, latent_dim, hidden_dim):
        super(LatentODEFunc, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, latent_dim)
        )

    def forward(self, t, x):
        """Time derivative of the latent state"""
        return self.net(x)

In [20]:
class Encoder(nn.Module):
    """Encodes the observed data into initial latent state"""
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(Encoder, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, latent_dim * 2)  # For mean and logvar

    def forward(self, x, time):
        """
        x: [batch_size, seq_len, input_dim]
        time: [batch_size, seq_len]
        """
        # Sort by time
        _, indices = torch.sort(time, dim=1)
        batch_size, seq_len = x.size(0), x.size(1)
        sorted_indices = indices.unsqueeze(-1).expand(-1, -1, x.size(-1))
        x_sorted = torch.gather(x, 1, sorted_indices)

        h, _ = self.lstm(x_sorted)
        h = h[:, -1, :]  # Take the last hidden state

        params = self.linear(h)
        mean, logvar = torch.chunk(params, 2, dim=1)

        # Reparameterization trick
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        z0 = mean + eps * std

        return z0, mean, logvar


In [21]:

class Decoder(nn.Module):
    """Decodes the latent state into observed data"""
    def __init__(self, latent_dim, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, z):
        """
        z: [batch_size, seq_len, latent_dim]
        """
        return self.net(z)


In [22]:
class LatentODE(nn.Module):
    """Complete Latent ODE model"""
    def __init__(self, input_dim, latent_dim, hidden_dim, device):
        super(LatentODE, self).__init__()
        self.encoder = Encoder(input_dim, hidden_dim, latent_dim)
        self.func = LatentODEFunc(latent_dim, hidden_dim)
        self.decoder = Decoder(latent_dim, hidden_dim, input_dim)
        self.latent_dim = latent_dim
        self.device = device

    def forward(self, x, time_points, target_time_points):
        """
        x: [batch_size, seq_len, input_dim]
        time_points: [batch_size, seq_len]
        target_time_points: times to evaluate the ODE at
        """
        batch_size = x.size(0)

        # Get initial latent state
        z0, mean, logvar = self.encoder(x, time_points)

        # Integrate the ODE
        target_time_points = target_time_points.to(self.device)
        z = odeint(self.func, z0, target_time_points, method='implicit_adams')
        z = z.permute(1, 0, 2)  # [batch_size, seq_len, latent_dim]

        # Decode
        x_pred = self.decoder(z)

        return x_pred, mean, logvar


In [23]:
# Model hyperparameters
input_dim = 3  # tx, ty, tz
latent_dim = 4
hidden_dim = 128
batch_size = 176
epochs = 2000
lr = 0.01

In [24]:
# Reshape for batched training (we'll use a batch size of 1 since we have one trajectory)
complete_dataset = TensorDataset(
    complete_features_tensor.unsqueeze(0),
    complete_time_tensor.unsqueeze(0)
)
complete_loader = DataLoader(complete_dataset, batch_size=1, shuffle=False)


In [25]:
# Create and train the model
model = LatentODE(input_dim, latent_dim, hidden_dim, device).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)


In [26]:
def loss_fn(pred, target, mean, logvar, mask=None):
    """
    Loss function with reconstruction loss and KL divergence
    """
    if mask is None:
        mask = torch.ones_like(target)

    # MSE reconstruction loss
    recon_loss = torch.sum(mask.unsqueeze(-1) * (pred - target)**2) / torch.sum(mask.unsqueeze(-1))

    # KL divergence
    kl_loss = -0.5 * torch.mean(1 + logvar - mean.pow(2) - logvar.exp())

    return recon_loss + 0.1 * kl_loss, recon_loss, kl_loss


In [27]:
# Training loop
print("Starting training...")
losses = []

for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0

    for batch_idx, (x_batch, t_batch) in enumerate(complete_loader):
        x_batch = x_batch.to(device)
        t_batch = t_batch.to(device)

        optimizer.zero_grad()

        # Forward pass
        x_pred, mean, logvar = model(x_batch, t_batch, complete_time_tensor.to(device))

        # Compute loss
        loss, recon_loss, kl_loss = loss_fn(x_pred, x_batch, mean, logvar, observed_mask_tensor.to(device))

        # Backward pass
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(complete_loader)
    losses.append(avg_loss)

    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.6f}")

print("Training completed!")

Starting training...
Epoch 20/2000, Loss: 0.447565


/home/chicken/Pytorch/lib/python3.10/site-packages/torchdiffeq/_impl/fixed_adams.py:220: UserWarning: Functional iteration did not converge. Solution may be incorrect.
  warnings.warn('Functional iteration did not converge. Solution may be incorrect.')


Epoch 40/2000, Loss: 0.292716
Epoch 60/2000, Loss: 0.301414
Epoch 80/2000, Loss: 0.269546
Epoch 100/2000, Loss: 0.244986
Epoch 120/2000, Loss: 0.271283
Epoch 140/2000, Loss: 0.247506
Epoch 160/2000, Loss: 0.262438
Epoch 180/2000, Loss: 0.287440
Epoch 200/2000, Loss: 0.270708
Epoch 220/2000, Loss: 0.253822
Epoch 240/2000, Loss: 0.253339
Epoch 260/2000, Loss: 0.256241
Epoch 280/2000, Loss: 0.249933
Epoch 300/2000, Loss: 0.255928
Epoch 320/2000, Loss: 0.248809
Epoch 340/2000, Loss: 0.234903
Epoch 360/2000, Loss: 0.232006
Epoch 380/2000, Loss: 0.257362
Epoch 400/2000, Loss: 0.282266
Epoch 420/2000, Loss: 0.262824
Epoch 440/2000, Loss: 0.246949
Epoch 460/2000, Loss: 0.234830
Epoch 480/2000, Loss: 0.265277
Epoch 500/2000, Loss: 0.224481
Epoch 520/2000, Loss: 0.245353
Epoch 540/2000, Loss: 0.225001
Epoch 560/2000, Loss: 0.240586
Epoch 580/2000, Loss: 0.278267
Epoch 600/2000, Loss: 0.220489
Epoch 620/2000, Loss: 0.255516
Epoch 640/2000, Loss: 0.223594
Epoch 660/2000, Loss: 0.198590
Epoch 680/2

In [28]:
# Evaluation and visualization
model.eval()
with torch.no_grad():
    x_complete = complete_features_tensor.unsqueeze(0).to(device)
    t_complete = complete_time_tensor.unsqueeze(0).to(device)

    # Get predictions for all time points
    x_pred, _, _ = model(x_complete, t_complete, complete_time_tensor.to(device))

    # Move predictions back to CPU and convert to numpy
    x_pred = x_pred.squeeze(0).cpu().numpy()

    # Inverse transform to get the original scale
    x_pred_original = feature_scaler.inverse_transform(x_pred)


In [29]:
# Compare the reconstructed trajectory with the original
plt.figure(figsize=(15, 10))
features = ['tx', 'ty', 'tz']

for i in range(3):
    plt.subplot(3, 1, i+1)

    # Original complete data
    plt.plot(complete_time, complete_features[:, i], 'b-', label='Dữ liệu gốc')

    # Incomplete data points
    plt.plot(incomplete_time, incomplete_features[:, i], 'ro', markersize=4, label='Dữ liệu được quan sát (Dữ liệu thiếu)')

    # Reconstructed trajectory
    plt.plot(complete_time, x_pred_original[:, i], 'g--', label='Dữ liệu tái tạo')

    plt.title(f'Đặc trưng: {features[i]}')
    plt.xlabel('Thời gian')
    plt.ylabel(features[i])
    plt.legend()

plt.tight_layout()
plt.savefig('uav_trajectory_reconstruction.png')
plt.close()

In [30]:
# Calculate reconstruction error
# First, find indices that exist in complete dataset but not in incomplete
missing_indices = np.where(observed_mask == 0)[0]
missing_original = complete_features[missing_indices]
missing_reconstructed = x_pred_original[missing_indices]

mse = np.mean((missing_original - missing_reconstructed) ** 2)
mae = np.mean(np.abs(missing_original - missing_reconstructed))

print(f"Missing points reconstruction - MSE: {mse:.6f}, MAE: {mae:.6f}")


Missing points reconstruction - MSE: 1.323392, MAE: 0.762658


In [31]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

In [32]:
# Calculate error metrics for each dimension
for i in range(3):
    mse_dim = np.mean((missing_original[:, i] - missing_reconstructed[:, i]) ** 2)
    mae_dim = np.mean(np.abs(missing_original[:, i] - missing_reconstructed[:, i]))
    print(f"Dimension {features[i]} - MSE: {mse_dim:.6f}, MAE: {mae_dim:.6f}")


Dimension tx - MSE: 2.694606, MAE: 1.267845
Dimension ty - MSE: 1.274157, MAE: 0.987514
Dimension tz - MSE: 0.001413, MAE: 0.032613


In [33]:
y_true_missing = missing_original
y_pred_missing = missing_reconstructed

# Dữ liệu thực tế và dự đoán trên toàn bộ quỹ đạo
y_true_all = complete_features
y_pred_all = x_pred_original

In [34]:
# --- 2. R-squared (R^2) ---
if y_true_missing.shape[0] > 0:
    r2_total_missing = r2_score(y_true_missing, y_pred_missing, multioutput='uniform_average') # Tính R^2 trung bình cho các chiều
    print(f"\nTổng R^2 (trên điểm bị thiếu, trung bình các chiều): {r2_total_missing:.6f}")

    for i in range(y_true_missing.shape[1]):
        # Kiểm tra để tránh lỗi nếu một chiều có phương sai bằng 0 (ít xảy ra với dữ liệu thực)
        if np.var(y_true_missing[:, i]) == 0 and np.var(y_pred_missing[:, i]) == 0 and np.mean(y_true_missing[:, i]) == np.mean(y_pred_missing[:, i]):
            r2_dim_missing = 1.0 # Dự đoán hoàn hảo cho một hằng số
        elif np.var(y_true_missing[:, i]) == 0:
             r2_dim_missing = 0.0 # Không thể giải thích phương sai nếu không có phương sai
        else:
            r2_dim_missing = r2_score(y_true_missing[:, i], y_pred_missing[:, i])
        print(f"R^2 cho chiều {features[i]} (trên điểm bị thiếu): {r2_dim_missing:.6f}")
else:
    print("\nKhông có điểm dữ liệu nào bị thiếu để tính R^2.")


Tổng R^2 (trên điểm bị thiếu, trung bình các chiều): 0.703817
R^2 cho chiều tx (trên điểm bị thiếu): 0.594658
R^2 cho chiều ty (trên điểm bị thiếu): 0.853192
R^2 cho chiều tz (trên điểm bị thiếu): 0.663602


In [35]:
# --- 4. Directional Accuracy (DA) ---
def directional_accuracy_func(y_true_da, y_pred_da):
    y_true_da, y_pred_da = np.asarray(y_true_da), np.asarray(y_pred_da)
    if len(y_true_da) < 2:
        return np.nan # Cần ít nhất 2 điểm để so sánh hướng

    true_diff = np.diff(y_true_da)
    pred_diff = np.diff(y_pred_da)

    true_direction = np.sign(true_diff)
    pred_direction = np.sign(pred_diff)

    # Xử lý trường hợp cả hai diff đều là 0 (không đổi hướng) là một dự đoán đúng
    correct_direction = (true_direction == pred_direction)
    return np.mean(correct_direction) * 100

In [36]:
print("\n--- Directional Accuracy (trên các điểm bị thiếu, nếu theo thứ tự thời gian) ---")
# Lưu ý: DA trên các điểm bị thiếu chỉ có ý nghĩa nếu các điểm đó được sắp xếp theo thời gian
# và tạo thành một chuỗi có thể phân tích hướng.
if y_true_missing.shape[0] >= 2:
    da_missing_list = []
    for i in range(y_true_missing.shape[1]):
        da_dim_missing = directional_accuracy_func(y_true_missing[:, i], y_pred_missing[:, i])
        if not np.isnan(da_dim_missing):
            da_missing_list.append(da_dim_missing)
        print(f"DA cho chiều {features[i]} (trên điểm bị thiếu): {da_dim_missing:.2f}%" if not np.isnan(da_dim_missing) else f"DA cho chiều {features[i]} (trên điểm bị thiếu): Không đủ dữ liệu")

    if da_missing_list:
        print(f"DA trung bình tổng thể (trên điểm bị thiếu, cho các chiều tính được): {np.mean(da_missing_list):.2f}%")
    else:
        print("DA trung bình tổng thể (trên điểm bị thiếu): Không thể tính.")
else:
    print("Không đủ điểm dữ liệu bị thiếu để tính Directional Accuracy.")


--- Directional Accuracy (trên các điểm bị thiếu, nếu theo thứ tự thời gian) ---
DA cho chiều tx (trên điểm bị thiếu): 69.77%
DA cho chiều ty (trên điểm bị thiếu): 93.02%
DA cho chiều tz (trên điểm bị thiếu): 67.44%
DA trung bình tổng thể (trên điểm bị thiếu, cho các chiều tính được): 76.74%


In [37]:
# --- R-squared (R^2) trên toàn bộ quỹ đạo ---
r2_total_all = r2_score(y_true_all, y_pred_all, multioutput='uniform_average')
print(f"\nTổng R^2 (toàn bộ quỹ đạo, trung bình các chiều): {r2_total_all:.6f}")
for i in range(y_true_all.shape[1]):
    if np.var(y_true_all[:, i]) == 0 and np.var(y_pred_all[:, i]) == 0 and np.mean(y_true_all[:, i]) == np.mean(y_pred_all[:, i]):
        r2_dim_all = 1.0
    elif np.var(y_true_all[:, i]) == 0:
        r2_dim_all = 0.0
    else:
        r2_dim_all = r2_score(y_true_all[:, i], y_pred_all[:, i])
    print(f"R^2 cho chiều {features[i]} (toàn bộ quỹ đạo): {r2_dim_all:.6f}")


Tổng R^2 (toàn bộ quỹ đạo, trung bình các chiều): 0.676473
R^2 cho chiều tx (toàn bộ quỹ đạo): 0.549990
R^2 cho chiều ty (toàn bộ quỹ đạo): 0.897744
R^2 cho chiều tz (toàn bộ quỹ đạo): 0.581685


In [38]:
# --- Directional Accuracy (DA) trên toàn bộ quỹ đạo ---
print("\n--- Directional Accuracy (toàn bộ quỹ đạo) ---")
if y_true_all.shape[0] >= 2:
    da_all_list = []
    for i in range(y_true_all.shape[1]):
        da_dim_all = directional_accuracy_func(y_true_all[:, i], y_pred_all[:, i])
        if not np.isnan(da_dim_all):
            da_all_list.append(da_dim_all)
        print(f"DA cho chiều {features[i]} (toàn bộ quỹ đạo): {da_dim_all:.2f}%" if not np.isnan(da_dim_all) else f"DA cho chiều {features[i]} (toàn bộ quỹ đạo): Không đủ dữ liệu")

    if da_all_list:
        print(f"DA trung bình tổng thể (toàn bộ quỹ đạo, cho các chiều tính được): {np.mean(da_all_list):.2f}%")
    else:
        print("DA trung bình tổng thể (toàn bộ quỹ đạo): Không thể tính.")
else:
    print("Không đủ điểm dữ liệu trên toàn bộ quỹ đạo để tính Directional Accuracy.")


--- Directional Accuracy (toàn bộ quỹ đạo) ---
DA cho chiều tx (toàn bộ quỹ đạo): 81.28%
DA cho chiều ty (toàn bộ quỹ đạo): 89.04%
DA cho chiều tz (toàn bộ quỹ đạo): 58.90%
DA trung bình tổng thể (toàn bộ quỹ đạo, cho các chiều tính được): 76.41%


In [39]:
# --- 3. Mean Absolute Percentage Error (MAPE) ---
def mape_func(y_true_m, y_pred_m, epsilon=1e-8): # Thêm epsilon để tránh chia cho 0 tuyệt đối
    y_true_m, y_pred_m = np.asarray(y_true_m), np.asarray(y_pred_m)

    # Xử lý trường hợp y_true_m bằng 0
    # Ở đây, chúng ta sẽ chỉ tính MAPE cho các điểm mà y_true_m khác 0.
    # Hoặc có thể thay thế y_true_m == 0 bằng một giá trị rất nhỏ (epsilon) nếu muốn bao gồm chúng
    # nhưng điều này có thể làm MAPE rất lớn nếu y_pred_m khác biệt đáng kể.

    mask = y_true_m != 0
    if not np.any(mask): # Nếu tất cả y_true_m đều bằng 0 (hoặc bị mask hết)
         return np.nan # Không thể tính MAPE

    # Chỉ tính trên các phần tử không bị mask
    y_true_masked = y_true_m[mask]
    y_pred_masked = y_pred_m[mask]

    # Nếu sau khi mask không còn phần tử nào (trường hợp hiếm nếu ban đầu có dữ liệu)
    if y_true_masked.shape[0] == 0:
        return np.nan

    return np.mean(np.abs((y_true_masked - y_pred_masked) / y_true_masked)) * 100

In [40]:
# MAPE trên toàn bộ quỹ đạo tái tạo
print("\nMAPE trên toàn bộ quỹ đạo tái tạo:")
if y_true_all.shape[0] > 0:
    mape_all_list = []
    for i in range(y_true_all.shape[1]):
        mape_dim_all = mape_func(y_true_all[:, i], y_pred_all[:, i])
        if not np.isnan(mape_dim_all):
            mape_all_list.append(mape_dim_all)
        print(f"MAPE cho chiều {features[i]} (toàn bộ quỹ đạo): {mape_dim_all:.2f}%" if not np.isnan(mape_dim_all) else f"MAPE cho chiều {features[i]} (toàn bộ quỹ đạo): Không thể tính (y_true có thể chứa giá trị 0)")

    if mape_all_list:
        print(f"MAPE trung bình tổng thể (toàn bộ quỹ đạo, cho các chiều tính được): {np.mean(mape_all_list):.2f}%")
    else:
        print("MAPE trung bình tổng thể (toàn bộ quỹ đạo): Không thể tính.")
else:
    print("Không có dữ liệu trên toàn bộ quỹ đạo để tính MAPE.")


MAPE trên toàn bộ quỹ đạo tái tạo:
MAPE cho chiều tx (toàn bộ quỹ đạo): 15.60%
MAPE cho chiều ty (toàn bộ quỹ đạo): 3.49%
MAPE cho chiều tz (toàn bộ quỹ đạo): 0.26%
MAPE trung bình tổng thể (toàn bộ quỹ đạo, cho các chiều tính được): 6.45%


In [41]:
# MAPE trên các điểm bị thiếu
print("\nMAPE trên các điểm bị thiếu:")
if y_true_missing.shape[0] > 0:
    mape_missing_list = []
    for i in range(y_true_missing.shape[1]):
        mape_dim_missing = mape_func(y_true_missing[:, i], y_pred_missing[:, i])
        if not np.isnan(mape_dim_missing):
            mape_missing_list.append(mape_dim_missing)
        print(f"MAPE cho chiều {features[i]} (trên điểm bị thiếu): {mape_dim_missing:.2f}%" if not np.isnan(mape_dim_missing) else f"MAPE cho chiều {features[i]} (trên điểm bị thiếu): Không thể tính (y_true có thể chứa giá trị 0 hoặc không có điểm)")

    if mape_missing_list:
        print(f"MAPE trung bình tổng thể (trên điểm bị thiếu, cho các chiều tính được): {np.mean(mape_missing_list):.2f}%")
    else:
        print("MAPE trung bình tổng thể (trên điểm bị thiếu): Không thể tính.")
else:
    print("Không có điểm dữ liệu nào bị thiếu để tính MAPE.")


MAPE trên các điểm bị thiếu:
MAPE cho chiều tx (trên điểm bị thiếu): 13.58%
MAPE cho chiều ty (trên điểm bị thiếu): 4.25%
MAPE cho chiều tz (trên điểm bị thiếu): 0.26%
MAPE trung bình tổng thể (trên điểm bị thiếu, cho các chiều tính được): 6.03%


In [42]:
# Create a more detailed comparison plot for missing points
plt.figure(figsize=(15, 10))
for i in range(3):
    plt.subplot(3, 1, i+1)

    # Original values at missing points
    plt.plot(complete_time[missing_indices], missing_original[:, i], 'bx', markersize=6, label='Dữ liệu gốc (Dữ liệu bị thiếu)')

    # Reconstructed values at missing points
    plt.plot(complete_time[missing_indices], missing_reconstructed[:, i], 'go', markersize=4, label='Dữ liệu tái tạo')

    plt.title(f'Tái tạo dữ liệu bị thiếu: {features[i]}')
    plt.xlabel('Thời gian')
    plt.ylabel(features[i])
    plt.legend()

plt.tight_layout()
plt.savefig('missing_points_comparison.png')
plt.close()

In [43]:
# Plot training loss
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(losses) + 1), losses)
plt.title('Mất mát khi huấn luyện')
plt.xlabel('Epoch')
plt.ylabel('Mất mát')
plt.grid(True)
plt.savefig('training_loss.png')
plt.close()


In [44]:
# 3D trajectory visualization
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Original complete trajectory
ax.plot3D(complete_features[:, 0], complete_features[:, 1], complete_features[:, 2],
         'b-', linewidth=1, label='Dữ liệu gốc')

# Observed points
ax.scatter(incomplete_features[:, 0], incomplete_features[:, 1], incomplete_features[:, 2],
          c='r', s=30, label='Điểm dữ liệu được quan sát')

# Reconstructed trajectory
ax.plot3D(x_pred_original[:, 0], x_pred_original[:, 1], x_pred_original[:, 2],
         'g--', linewidth=2, label='Dữ liệu được tái tạo')

ax.set_xlabel('tx')
ax.set_ylabel('ty')
ax.set_zlabel('tz')
ax.set_title('Tái tạo quỹ đạo 3D của UAV')
ax.legend()

plt.tight_layout()
plt.savefig('uav_3d_trajectory.png')
plt.close()

In [45]:
# Save the reconstructed trajectory to CSV
reconstructed_df = pd.DataFrame({
    'timestamp': complete_time,
    'tx': x_pred_original[:, 0],
    'ty': x_pred_original[:, 1],
    'tz': x_pred_original[:, 2]
})
reconstructed_df.to_csv('uav_reconstructed_trajectory.csv', index=False)
print("Reconstructed trajectory saved to 'uav_reconstructed_trajectory.csv'")

Reconstructed trajectory saved to 'uav_reconstructed_trajectory.csv'
